In [1]:
print("Hello")

Hello


## Step 1 : Data Exploration and Loading


In [62]:
#import dependencies

import pandas as pd
import os

# mysql toolkit
import pymysql # this will work as adapter
from sqlalchemy import create_engine
from dotenv import load_dotenv

In [13]:
print(pd.__version__)

2.3.0


In [23]:
#encoding_error used to ignore the error on lines
df = pd.read_csv("Walmart.csv", encoding='ISO-8859-1')

df.shape
df.head()

,invoice_id,Branch,City,category,unit_price,quantity,date,time,payment_method,rating,profit_margin
0,1,WALM003,San Antonio,Health and beauty,$74.69,7.0,05/01/19,13:08:00,Ewallet,9.1,0.48
1,2,WALM048,Harlingen,Electronic accessories,$15.28,5.0,08/03/19,10:29:00,Cash,9.6,0.48
2,3,WALM067,Haltom City,Home and lifestyle,$46.33,7.0,03/03/19,13:23:00,Credit card,7.4,0.33
3,4,WALM064,Bedford,Health and beauty,$58.22,8.0,27/01/19,20:33:00,Ewallet,8.4,0.33
4,5,WALM013,Irving,Sports and travel,$86.31,7.0,08/02/19,10:37:00,Ewallet,5.3,0.48


In [24]:
df.describe() # stats 

,invoice_id,quantity,rating,profit_margin
count,10051.000000,10020.000000,10051.000000,10051.000000
mean,5025.741220,2.353493,5.825659,0.393791
std,2901.174372,1.602658,1.763991,0.090669
min,1.000000,1.000000,3.000000,0.180000
25%,2513.500000,1.000000,4.000000,0.330000
50%,5026.000000,2.000000,6.000000,0.330000
75%,7538.500000,3.000000,7.000000,0.480000
max,10000.000000,10.000000,10.000000,0.570000


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10051 entries, 0 to 10050
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_id      10051 non-null  int64  
 1   Branch          10051 non-null  object 
 2   City            10051 non-null  object 
 3   category        10051 non-null  object 
 4   unit_price      10020 non-null  object 
 5   quantity        10020 non-null  float64
 6   date            10051 non-null  object 
 7   time            10051 non-null  object 
 8   payment_method  10051 non-null  object 
 9   rating          10051 non-null  float64
 10  profit_margin   10051 non-null  float64
dtypes: float64(3), int64(1), object(7)
memory usage: 863.9+ KB


In [27]:
df.duplicated().sum() # counts all the dupliactes

np.int64(51)

In [28]:
#total number of null
df.isnull().sum()

invoice_id         0
Branch             0
City               0
category           0
unit_price        31
quantity          31
date               0
time               0
payment_method     0
rating             0
profit_margin      0
dtype: int64

In [31]:
#deal with duplicates
df.drop_duplicates(inplace=True)

df.duplicated().sum() # after removing duplicates

np.int64(0)

In [33]:
df.shape  # after removing duplicates

(10000, 11)

In [35]:
# deal with null values
#method : drop the rows if missing values
df.dropna(inplace=True)

df.isnull().sum() # verify

invoice_id        0
Branch            0
City              0
category          0
unit_price        0
quantity          0
date              0
time              0
payment_method    0
rating            0
profit_margin     0
dtype: int64

In [36]:
df.shape

(9969, 11)

In [37]:
#convert column
df.dtypes

invoice_id          int64
Branch             object
City               object
category           object
unit_price         object
quantity          float64
date               object
time               object
payment_method     object
rating            float64
profit_margin     float64
dtype: object

In [40]:
df['unit_price'] = df['unit_price'].str.replace('$','').astype(float)

df.head()

<bound method NDFrame.head of       invoice_id   Branch         City                category  unit_price  \
0              1  WALM003  San Antonio       Health and beauty       74.69   
1              2  WALM048    Harlingen  Electronic accessories       15.28   
2              3  WALM067  Haltom City      Home and lifestyle       46.33   
3              4  WALM064      Bedford       Health and beauty       58.22   
4              5  WALM013       Irving       Sports and travel       86.31   
...          ...      ...          ...                     ...         ...   
9995        9996  WALM056      Rowlett     Fashion accessories       37.00   
9996        9997  WALM030   Richardson      Home and lifestyle       58.00   
9997        9998  WALM050     Victoria     Fashion accessories       52.00   
9998        9999  WALM032        Tyler      Home and lifestyle       79.00   
9999       10000  WALM069     Rockwall     Fashion accessories       62.00   

      quantity      date      tim

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9969 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_id      9969 non-null   int64  
 1   Branch          9969 non-null   object 
 2   City            9969 non-null   object 
 3   category        9969 non-null   object 
 4   unit_price      9969 non-null   float64
 5   quantity        9969 non-null   float64
 6   date            9969 non-null   object 
 7   time            9969 non-null   object 
 8   payment_method  9969 non-null   object 
 9   rating          9969 non-null   float64
 10  profit_margin   9969 non-null   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 934.6+ KB


In [42]:
# shows column in csv file
df.columns

Index(['invoice_id', 'Branch', 'City', 'category', 'unit_price', 'quantity',
       'date', 'time', 'payment_method', 'rating', 'profit_margin'],
      dtype='object')

In [43]:
# create new column

df['total'] = df['unit_price']*df['quantity']

df.head()

,invoice_id,Branch,City,category,unit_price,quantity,date,time,payment_method,rating,profit_margin,total
0,1,WALM003,San Antonio,Health and beauty,74.69,7.0,05/01/19,13:08:00,Ewallet,9.1,0.48,522.83
1,2,WALM048,Harlingen,Electronic accessories,15.28,5.0,08/03/19,10:29:00,Cash,9.6,0.48,76.40
2,3,WALM067,Haltom City,Home and lifestyle,46.33,7.0,03/03/19,13:23:00,Credit card,7.4,0.33,324.31
3,4,WALM064,Bedford,Health and beauty,58.22,8.0,27/01/19,20:33:00,Ewallet,8.4,0.33,465.76
4,5,WALM013,Irving,Sports and travel,86.31,7.0,08/02/19,10:37:00,Ewallet,5.3,0.48,604.17


In [ ]:
# mysql 
# host = localhost
# port = 3306
# user = root
# password = "your_password"



In [46]:
df.to_csv('walmart_clean.csv', index = False)

In [63]:
#connect this project to mysql

load_dotenv()

db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASS")
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")

engine_mysql = create_engine(f'mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}')
try:
    engine_mysql
    print("Connection to databse is successfull")
except:
    print("Connection to database is failed")

Connection to databse is successfull


In [57]:
# create table into walmartsales_db databse named walmart

df.to_sql(name='walmart', con=engine_mysql, if_exists='append', index = False)

9969

In [58]:
# make al column names into lowercase
df.columns = df.columns.str.lower()